In [0]:
CREATE OR REPLACE VIEW view_dashboard_valuation AS
WITH inputs AS (
    -- User Inputs (Simulated with Widgets in Dashboard)
    SELECT 
        10000 as investment_amount, -- :investment_amount
        5 as years_horizon          -- :years_horizon
)
SELECT 
    s.symbol,
    s.valuation_timestamp,
    s.close_price,
    
    -- Display WACC safely
    round(s.wacc, 4) as wacc,
    
    -- 1. Treasury Benchmark
    round(i.investment_amount * power(1.0425, i.years_horizon), 2) as treasury_future_value,
    
    -- 2. Stock Projection
    -- Logic: We strictly filter out bad WACC below, so this calculation is now safe.
    round(i.investment_amount * power(1 + s.wacc, i.years_horizon), 2) as stock_future_value,

    -- 3. The "Alpha"
    round(
        (i.investment_amount * power(1 + s.wacc, i.years_horizon)) - 
        (i.investment_amount * power(1.0425, i.years_horizon)), 
    2) as projected_alpha,

    -- 4. Human Intel
    CASE 
        WHEN s.wacc > 0.12 THEN 'High Cost of Capital (Risky Bet)'
        WHEN s.wacc < 0.06 THEN 'Cheap Capital (Stable Giant)'
        ELSE 'Balanced Risk'
    END as capital_intel

FROM delta.`/Volumes/workspace/default/storage/serving/valuation_dashboard_v5` s
CROSS JOIN inputs i

WHERE 
    -- SAFETY FILTER: Drop rows where WACC is broken
    s.wacc IS NOT NULL 
    AND NOT isnan(s.wacc) 
    AND s.wacc > 0 -- Sanity check: WACC should generally be positive

ORDER BY s.valuation_timestamp DESC;